# Daten untersuchen mit Pandas und Python für Fortgeschrittene

### Daten aus der CSV-Datei lesen

Nun benutzen wir pandas um die [CSV-Datei](https://de.wikipedia.org/wiki/CSV_(Dateiformat)) einzulesen. Damit das funktioniert muss vorher noch das pandas-Paket mit dem `import`-Befehl unter dem Namen `pd` verfügbar gemacht werden.

Dabei wird ein sogenannter [DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) mit dem Namen `nba` erzeugt. Vereinfacht kann man sich darunter eine Tabelle mit Spalten und Zeilen vorstellen. DataFrames sind **die** zentralen Datentypen in pandas.   

Mit dem Befehl `type()` kann der Datentyp eines jeden Python-Objekts ausgegeben werden. 

In [212]:
import pandas as pd
import numpy as np
nba = pd.read_csv("nba_all_elo.csv")
type(nba)
# Expected:
# <class 'pandas.core.frame.DataFrame'>

pandas.core.frame.DataFrame

In [213]:
nba.head()

,gameorder,game_id,lg_id,_iscopy,year_id,date_game,seasongame,is_playoffs,team_id,fran_id,...,win_equiv,opp_id,opp_fran,opp_pts,opp_elo_i,opp_elo_n,game_location,game_result,forecast,notes
0,1,194611010TRH,NBA,0,1947,11/1/1946,1,0,TRH,Huskies,...,40.294830,NYK,Knicks,68,1300.0000,1306.7233,H,L,0.640065,NaN
1,1,194611010TRH,NBA,1,1947,11/1/1946,1,0,NYK,Knicks,...,41.705170,TRH,Huskies,66,1300.0000,1293.2767,A,W,0.359935,NaN
2,2,194611020CHS,NBA,0,1947,11/2/1946,1,0,CHS,Stags,...,42.012257,NYK,Knicks,47,1306.7233,1297.0712,H,W,0.631101,NaN
3,2,194611020CHS,NBA,1,1947,11/2/1946,2,0,NYK,Knicks,...,40.692783,CHS,Stags,63,1300.0000,1309.6521,A,L,0.368899,NaN
4,3,194611020DTF,NBA,0,1947,11/2/1946,1,0,DTF,Falcons,...,38.864048,WSC,Capitols,50,1300.0000,1320.3811,H,L,0.640065,NaN


# Aufgabe: Finals
![finals](./finals.png)

Seit 2005 werden die Playoffs, das Finalturnier der NBA, in einem Wettbewerb im Best-of-Seven Moduls gespielt.
Eine Finalserie hat also zwischen 4 und 7 Spielen zwischen einer Spielpaarung. Dabei werden die ersten vier Spiele je 2 Spiele am Heimort der ersten Mannschaft und anschließend zwei Spiele am Heimort der anderen Mannschaft gespielt. Danach wird der Spielort pro Spiel gewechselt. Folglich startet eine Mannschaft mit zwei Auswärtsspielen in eines solche Finalserie. Siehe: https://en.wikipedia.org/wiki/NBA_playoffs

![finals](./finals_PHX_DAL.png)



Da im Sport Erfahrungen, Binsenweisheiten und andere Mutmaßungen sehr weit verbreitet ist können man sich folgenden Ausspruch vorstellen:

>"Das Auswärtsteam wird eine Serie eher gewinnen wenn Sie das erste Auswärtsspiel (Spiel 1 der Serie) gewinnt und das zweite verliert als anders herum."

Da wir die Daten zu allen Spielen haben lässt sich das ja nun eigentlich sehr einfach nachprüfen.

In den nächsten beiden Einheiten werden wird die Daten soweit aufbereiten um diese Hypothese zu testen. Dazu müssen wir nun erst einmal die Serien in den Daten nachbilden.


## Vorbereitung

1. Alle Playoffspiele ab 2015 extrahieren

Dazu Filter auf `year_id` und `is_playoffs` verwenden:

In [214]:
playoffs2005= nba[
    (nba["is_playoffs"] == 1) & 
    (nba["year_id"] >= 2005) # Datum von
    ]
playoffs2005

,gameorder,game_id,lg_id,_iscopy,year_id,date_game,seasongame,is_playoffs,team_id,fran_id,...,win_equiv,opp_id,opp_fran,opp_pts,opp_elo_i,opp_elo_n,game_location,game_result,forecast,notes
100346,50174,200504230BOS,NBA,0,2005,4/23/2005,83,1,BOS,Celtics,...,44.004639,IND,Pacers,82,1564.4844,1550.8414,H,W,0.567768,NaN
100347,50174,200504230BOS,NBA,1,2005,4/23/2005,83,1,IND,Pacers,...,46.631470,BOS,Celtics,102,1511.8661,1525.5090,A,L,0.432232,NaN
100348,50175,200504230DAL,NBA,0,2005,4/23/2005,83,1,DAL,Mavericks,...,58.357548,HOU,Rockets,98,1669.1071,1686.3287,H,L,0.668070,NaN
100349,50175,200504230DAL,NBA,1,2005,4/23/2005,83,1,HOU,Rockets,...,59.455257,DAL,Mavericks,86,1690.6172,1673.3955,A,W,0.331930,NaN
100350,50176,200504230DET,NBA,0,2005,4/23/2005,83,1,DET,Pistons,...,53.457451,PHI,Sixers,85,1500.2852,1493.7009,H,W,0.772712,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126309,63155,201506110CLE,NBA,0,2015,6/11/2015,100,1,CLE,Cavaliers,...,60.309792,GSW,Warriors,103,1790.9591,1809.9791,H,L,0.546572,NaN
126310,63156,201506140GSW,NBA,0,2015,6/14/2015,102,1,GSW,Warriors,...,68.013329,CLE,Cavaliers,91,1704.3949,1700.7391,H,W,0.765565,NaN
126311,63156,201506140GSW,NBA,1,2015,6/14/2015,101,1,CLE,Cavaliers,...,60.010067,GSW,Warriors,104,1809.9791,1813.6349,A,L,0.234435,NaN
126312,63157,201506170CLE,NBA,0,2015,6/16/2015,102,1,CLE,Cavaliers,...,59.290245,GSW,Warriors,105,1813.6349,1822.2881,H,L,0.481450,NaN


2. Alle Series identifizieren und zu den Spielen als neue Spalte `series` zurordnen. Inhalt soll sein teamid1_teamid2 also z.B. `BOS_LAL` für eine Serie zwischen Boston und Los Angeles. 

Mit der loc-Funktion alle Zeilen (:) und due neue Spalte `series` selektieren. Inhalt der neuen SPalte wird `team_id` verkettet mit `opp_id`.

In [215]:
playoffs2005.loc[:,"series"] = playoffs2005["team_id"].str.cat(playoffs2005["opp_id"], sep ="_")

c:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


3. Alle Spiele innerhalb einer Series nummerieren. Tipp: Benutzen Sie die Funktion rank() bzw. groupby() und  GroupBy.rank()

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rank.html bzw.
https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.GroupBy.rank.html#pandas.core.groupby.GroupBy.rank 

Datum konvertieren.

In [216]:
playoffs2005["date_game"] = pd.to_datetime(playoffs2005["date_game"])

c:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Kopie anlegen und Spiele pro Serie und Jahr (`groupby()`) nach Datum (`date_game`) nummerieren (`rank()`).

In [217]:
playoffs2005_grouped = playoffs2005
playoffs2005_grouped["series_game"]=playoffs2005.groupby(["year_id","series"])["date_game"].rank()


c:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Zum Test zwei Serien betrachten: Dallas vs Houston 2005 und 2015

In [218]:
playoffs2005_grouped[playoffs2005_grouped["series"] == "DAL_HOU"]

,gameorder,game_id,lg_id,_iscopy,year_id,date_game,seasongame,is_playoffs,team_id,fran_id,...,opp_fran,opp_pts,opp_elo_i,opp_elo_n,game_location,game_result,forecast,notes,series,series_game
100348,50175,200504230DAL,NBA,0,2005,2005-04-23,83,1,DAL,Mavericks,...,Rockets,98,1669.1071,1686.3287,H,L,0.668070,NaN,DAL_HOU,1.0
100364,50183,200504250DAL,NBA,0,2005,2005-04-25,84,1,DAL,Mavericks,...,Rockets,113,1686.3287,1692.7972,H,L,0.622741,NaN,DAL_HOU,2.0
100379,50190,200504280HOU,NBA,1,2005,2005-04-28,85,1,DAL,Mavericks,...,Rockets,102,1692.7972,1683.3229,A,W,0.326388,NaN,DAL_HOU,3.0
100393,50197,200504300HOU,NBA,1,2005,2005-04-30,86,1,DAL,Mavericks,...,Rockets,93,1683.3229,1674.3434,A,W,0.350808,NaN,DAL_HOU,4.0
100407,50204,200505020DAL,NBA,0,2005,2005-05-02,87,1,DAL,Mavericks,...,Rockets,100,1674.3434,1670.7961,H,W,0.654569,NaN,DAL_HOU,5.0
100423,50212,200505050HOU,NBA,1,2005,2005-05-05,88,1,DAL,Mavericks,...,Rockets,101,1670.7961,1681.7832,A,L,0.384316,NaN,DAL_HOU,6.0
100430,50216,200505070DAL,NBA,0,2005,2005-05-07,89,1,DAL,Mavericks,...,Rockets,76,1681.7832,1663.4641,H,W,0.634954,NaN,DAL_HOU,7.0
126156,63079,201504180HOU,NBA,1,2015,2015-04-18,83,1,DAL,Mavericks,...,Rockets,118,1639.3665,1644.2307,A,L,0.267220,NaN,DAL_HOU,1.0
126174,63088,201504210HOU,NBA,1,2015,2015-04-21,84,1,DAL,Mavericks,...,Rockets,111,1644.2307,1649.4286,A,L,0.256398,NaN,DAL_HOU,2.0
126190,63096,201504240DAL,NBA,0,2015,2015-04-24,85,1,DAL,Mavericks,...,Rockets,130,1649.4286,1654.3431,H,L,0.506668,NaN,DAL_HOU,3.0


4. Die Anzahl der gewonnen Spiele in der Serie berechnen. in die Spalten `num_series_wins` bzw- `opp_num_series_wins` eintragen. Danach alle Ergebnisse der Serien finden und die `team_id` der Siegermannschaft als neue Spalte `series_winner` eintragen 

Per `groupby()` und `count()` die Anzahl der Siege und Niederlagen zählen. Achtung: Immer aus Perspektive der Heimmannschaft. Spalten anschließend entsprechend benennen.

In [219]:
serieswins = playoffs2005[playoffs2005["game_result"]=='W'].groupby(["year_id","series"])["date_game"].count()
serieswins.name = 'num_series_wins'

In [220]:
serieslosses = playoffs2005[playoffs2005["game_result"]=='L'].groupby(["year_id","series"])["date_game"].count()
serieslosses.name = 'opp_series_wins'

Die Funktion `merge()` verbindet zwei DataFrames anhand einer oder mehreren Schlüsselspalten. Hier werden alle Spiele mit den zuvor ermittelten Kennzahlen zu Siegen und Niederlagen verbunden. Zu jedem Spiel gibt es also nun eine Information zu der übergeordneten Series.

In [221]:
playoffs2005_merged = pd.merge(left=playoffs2005_grouped,right=serieswins,on=["year_id","series"])
playoffs2005_merged = pd.merge(left=playoffs2005_merged,right=serieslosses,on=["year_id","series"])

Die neue Spalte `series_winner` wird abhängig vom Sieg/Niederlagen-Verhältnis entweder mit dem Heim oder dem Auswärtsteam gefüllt.

In [222]:
playoffs2005_merged.loc[playoffs2005_merged['num_series_wins'] > playoffs2005_merged['opp_series_wins'], 'series_winner'] = playoffs2005_merged["team_id"]
playoffs2005_merged.loc[playoffs2005_merged['num_series_wins'] < playoffs2005_merged['opp_series_wins'], 'series_winner'] =  playoffs2005_merged["opp_id"]

Inhalte überprüfen:

In [223]:
playoffs2005_merged.tail(20)

,gameorder,game_id,lg_id,_iscopy,year_id,date_game,seasongame,is_playoffs,team_id,fran_id,...,opp_elo_n,game_location,game_result,forecast,notes,series,series_game,num_series_wins,opp_series_wins,series_winner
1606,63147,201505230HOU,NBA,1,2015,2015-05-23,95,1,GSW,Warriors,...,1643.4324,A,W,0.536737,NaN,GSW_HOU,3.0,4,1,GSW
1607,63149,201505250HOU,NBA,1,2015,2015-05-25,96,1,GSW,Warriors,...,1658.9934,A,L,0.599427,NaN,GSW_HOU,4.0,4,1,GSW
1608,63151,201505270GSW,NBA,0,2015,2015-05-27,97,1,GSW,Warriors,...,1654.6359,H,W,0.798222,NaN,GSW_HOU,5.0,4,1,GSW
1609,63143,201505190GSW,NBA,1,2015,2015-05-19,95,1,HOU,Rockets,...,1789.7388,A,L,0.222043,NaN,HOU_GSW,1.0,1,4,GSW
1610,63145,201505210GSW,NBA,1,2015,2015-05-21,96,1,HOU,Rockets,...,1791.2300,A,L,0.217323,NaN,HOU_GSW,2.0,1,4,GSW
1611,63147,201505230HOU,NBA,0,2015,2015-05-23,97,1,HOU,Rockets,...,1813.4543,H,L,0.463263,NaN,HOU_GSW,3.0,1,4,GSW
1612,63149,201505250HOU,NBA,0,2015,2015-05-25,98,1,HOU,Rockets,...,1797.8933,H,W,0.400573,NaN,HOU_GSW,4.0,1,4,GSW
1613,63151,201505270GSW,NBA,1,2015,2015-05-27,99,1,HOU,Rockets,...,1802.2509,A,L,0.201778,NaN,HOU_GSW,5.0,1,4,GSW
1614,63152,201506040GSW,NBA,1,2015,2015-06-04,97,1,CLE,Cavaliers,...,1806.2035,A,L,0.250779,NaN,CLE_GSW,1.0,2,4,GSW
1615,63153,201506070GSW,NBA,1,2015,2015-06-07,98,1,CLE,Cavaliers,...,1797.5032,A,W,0.242326,NaN,CLE_GSW,2.0,2,4,GSW
